# ----------------------------
# import zone

In [9]:
from loadTDMS import *
from fft_test import *
import numpy as np
import pandas as pd
# from dask import dataframe as dd
import matplotlib.pyplot as plt
import itertools
import os
from tqdm import tqdm

# -----------------------------
# 関数まとめzone

## ①extract_16384
### 生データ から2^14個取ってくる (中間データ①)

In [4]:
# AE_{num}.tdms -> AE_extract_{num}.csv
# 生データ から2^14個取ってくる

def extract_16384(num):
    load_path = rf"Z:\100 Raw_data\20221027---切削実験\AE\Cutting\AE_{num}.tdms"
    save_path = rf"Z:\200 Produced_data\20221027\extract\AE_extract_{num}.csv"
    d = load_tdms(load_path, ch_num=3)
    d = d[0][5000000:5000000+16384]
    pd.DataFrame(d).to_csv(save_path)

## ②FFT_withoutNorm
### FFT(中間データ②)

In [11]:
def FFT_withoutNorm(num):
    
    #データ読み込み
    d = pd.read_csv(rf'Z:\200 Produced_data\20221027\extract\AE_extract_{num}.csv')
    d = d['0']
    
    #FFT条件
    dt = 1/10**6
    split_t_r = 0.1 # 1つの枠で全体のどの割合のデータを分析するか。
    overlap = 0.5  # オーバーラップ率
    window_F = "hamming"  # 窓関数選択: hanning, hamming, blackman
    y_label = "amplitude"
    y_unit = "V"
    N = len(d)
    fq = np.linspace(0, N*dt, N)  # 周波数軸　linspace(開始,終了,分割数)
    

    #FFT実行
    fq_, amp_ = FFT_main(fq, d, dt, split_t_r, overlap, window_F)

    #fig保存ー正規化前
    output_FN = fr"Z:\200 Produced_data\20221027\FFT\fig\FFT_{num}"
    plot_FFT(fq_, amp_, output_FN, y_label, y_unit,num)

    #dBに変換してcsv保存ー正規化前
    amp_db = pd.DataFrame({'db': 20*np.log10(amp_)})
    output_csv = fr"Z:\200 Produced_data\20221027\FFT\csv\FFT_{num}.csv"
    amp_db.to_csv(output_csv, header=False, index=False)

## ③正規化
### FFTを正規化
### 学習（trarin）データ作成（中間データ③）

In [49]:
#学習データから最大最小を算出
#今回は2022年6月の200path-300path
def get_minmax(start,end):
    all_data = []
    for i in range(start,end+1):
        d = pd.read_csv(rf"Z:\200 Produced_data\20221027\FFT\csv\FFT_{i}.csv")
        d = d.to_numpy()
        d = np.ravel(d)
        all_data.append(d[:])
    all_data = np.array(all_data)
    all_data = all_data.ravel()
    min_d = all_data.min()
    max_d = all_data.max()
    return min_d, max_d

#正規化_minmax
def normalize_minmax(min_, max_, d):
    min_max_d = (d - min_) / (max_ - min_)
    return min_max_d    

# ----------------------------------------
# 実行zone

## extract_16384, FFT_withoutNorm

In [ ]:
%%time
for i in tqdm(range(1,170)):
    extract_16384(i)
    
%%time
for i in tqdm(range(1,170)):
    FFT_withoutNorm(i)

## get_minmax
RMSを見て、定常摩耗域を推定して、推定したpathの領域のminmaxを取る

In [ ]:
min_, max_ = get_minmax(200,300)

## normalize_minmax

In [ ]:
%%time
for i in tqdm(range(1,341)):
    d = np.loadtxt(rf"Z:\200 Produced_data\20221027\FFT\csv\FFT_{i}.csv", delimiter=',')  # 区切り文字としてカンマを指定
    d_norm = normalize_minmax(min_,max_,d)
    amp_db_norm = pd.DataFrame({'db': d_norm})
    output_csv = rf"Z:\200 Produced_data\20221027\FFT_Norm\csv\FFT_Norm_{i}.csv"
    amp_db_norm.to_csv(output_csv, header=False, index=False)

# -------------------------------------------------------------------------------------
# --------------------------------------------------------------------------------------
# 2021年11月

# 検証データ(test data)作成

# ①生データ から2^14個取ってくる
## 中間データ①

In [63]:
# AE_{num}.tdms -> AE_extract_{num}.csv
# 生データ から2^14個取ってくる

def extract_16384(num):
    load_path = rf"Z:\100 実験データ（RAW）\20211115---切削実験---AE+切削抵抗\AE\1-5000\OnlyCuttingData\AE_{num}.tdms"
    save_path = rf"Z:\200 実験データ加工\20211115---切削実験---AE+切削抵抗\TrainingData\extract\AE_extract_{num}.csv"
    d = load_tdms(load_path, ch_num=3)
    d = d[0][5000000:5000000+16384]
    pd.DataFrame(d).to_csv(save_path)

In [65]:
%%time
for i in range(1,242):
    extract_16384(i)

Wall time: 9min 41s


# ②正規化する前のFFT
## 中間データ②作成

In [66]:
def FFT_withoutNorm(num):
    
    #データ読み込み
    d = pd.read_csv(rf'Z:\200 実験データ加工\20211115---切削実験---AE+切削抵抗\TrainingData\extract\AE_extract_{num}.csv')
    d = d['0']
    
    #FFT条件
    dt = 1/10**6
    split_t_r = 0.1 # 1つの枠で全体のどの割合のデータを分析するか。
    overlap = 0.5  # オーバーラップ率
    window_F = "hamming"  # 窓関数選択: hanning, hamming, blackman
    y_label = "amplitude"
    y_unit = "V"
    N = len(d)
    fq = np.linspace(0, N*dt, N)  # 周波数軸　linspace(開始,終了,分割数)
    

    #FFT実行
    fq_, amp_ = FFT_main(fq, d, dt, split_t_r, overlap, window_F)

    #fig保存ー正規化前
    output_FN = rf"Z:\200 実験データ加工\20211115---切削実験---AE+切削抵抗\TrainingData\FFT\fig\FFT_{num}"
    plot_FFT(fq_, amp_, output_FN, y_label, y_unit,num)

    #dBに変換してcsv保存ー正規化前
    amp_db = pd.DataFrame({'db': 20*np.log10(amp_)})
    output_csv = rf"Z:\200 実験データ加工\20211115---切削実験---AE+切削抵抗\TrainingData\FFT\csv\FFT_{num}.csv"
    amp_db.to_csv(output_csv, header=False, index=False)

In [67]:
%%time
for i in range(1,242):
    FFT_withoutNorm(i)

Wall time: 2min 6s


# FFT(正規化）
## 検証（test）データ作成

In [68]:
#学習データから最大最小を算出
#今回は2022年6月の200path-300path
def get_minmax(start,end):
    all_data = []
    for i in range(start,end+1):
        d = pd.read_csv(rf"Z:\200 実験データ加工\20220630\TrainingData\FFT\csv\FFT_{i}.csv")
        d = d.to_numpy()
        d = np.ravel(d)
        all_data.append(d[:])
    all_data = np.array(all_data)
    all_data = all_data.ravel()
    min_d = all_data.min()
    max_d = all_data.max()
    return min_d, max_d

#正規化_minmax
def normalize_minmax(min_, max_, d):
    min_max_d = (d - min_) / (max_ - min_)
    return min_max_d

In [69]:
min_, max_ = get_minmax(200,300)

In [70]:
%%time
for i in range(1,242):
    d = np.loadtxt(rf"Z:\200 実験データ加工\20211115---切削実験---AE+切削抵抗\TrainingData\FFT\csv\FFT_{i}.csv", delimiter=',')  # 区切り文字としてカンマを指定
    d_norm = normalize_minmax(min_,max_,d)
    amp_db_norm = pd.DataFrame({'db': d_norm})
    output_csv = rf"Z:\200 実験データ加工\20211115---切削実験---AE+切削抵抗\TrainingData\FFT_Norm\csv\FFT_Norm_{i}.csv"
    amp_db_norm.to_csv(output_csv, header=False, index=False)

Wall time: 6.42 s
